

docker cp /home/yury/weatherAUS.csv bd_cont:/home/weatherAUS.csv

docker cp /home/yury/normalise.ipynb bd_cont:/home/normalise.ipynb

In [2]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.DoubleType

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.DoubleType


In [3]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("part-00000-dfc11f65-5287-4c73-be9f-d12bac5d8fff-c000.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [4]:
weather_org_df.registerTempTable("Aus_Rain_Data")

<font color='red'>
YURY =====================================      MY CODE STARTS     =====================================
 </font>

In [5]:
val df0 = spark.sql("select * from Aus_Rain_Data")

df0: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [6]:
val df1 = df0.withColumn("Pressure9amTmp", df0("Pressure9am").cast(DoubleType))
    .drop("Pressure9am")
    .withColumnRenamed("Pressure9amTmp", "Pressure9am")

val df2 = df1.withColumn("Pressure3pmTmp", df1("Pressure3pm").cast(DoubleType))
    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pmTmp", "Pressure3pm")

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]
df2: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [7]:
val min_P9am = df2.agg(min("Pressure9am") as "min_P9am").select(col("min_P9am")).first.getDouble(0)
val max_P9am = df2.agg(max("Pressure9am") as "max_P9am").select(col("max_P9am")).first.getDouble(0)

min_P9am: Double = 980.5
max_P9am: Double = 1041.0


In [8]:
val min_P3pm = df2.agg(min("Pressure3pm") as "min_P3pm").select(col("min_P3pm")).first.getDouble(0)
val max_P3pm = df2.agg(max("Pressure3pm") as "max_P3pm").select(col("max_P3pm")).first.getDouble(0)

min_P3pm: Double = 977.1
max_P3pm: Double = 1039.6


In [9]:
val minPRange = 970.0    //from the report
val maxPRange = 1050.0   //from the report

minPRange: Double = 970.0
maxPRange: Double = 1050.0


<font color='blue'>
\begin{equation}
  Rescaled_{9am} = \frac{Pressure_i - Pressure_{min9am}}{Pressure_{max9am} - Pressure_{min9am}} * (maxPressureRange - minPressureRange) + minPressureRange
\end{equation}
<font color='red'>

In [10]:
val df3 = df2.withColumn("Pressure3pmTmp", 
    (((df2("Pressure3pm") - min_P3pm) / (max_P3pm -min_P3pm)) * (maxPRange - minPRange)) + minPRange)
    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pmTmp", "Pressure3pm")

df3: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [11]:
val df4 = df3.withColumn("Pressure9amTmp", 
    (((df2("Pressure9am") - min_P9am) / (max_P9am -min_P9am)) * (maxPRange - minPRange)) + minPRange)
    .drop("Pressure9am")
    .withColumnRenamed("Pressure9amTmp", "Pressure9am")

df4: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [12]:
df4.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("normalised")

In [13]:
!ls /home/normalised

_SUCCESS  part-00000-d16d2c38-3b7a-4413-9e3e-6c8778709a3e-c000.csv



In [14]:
!mv /home/normalised/*.csv /home/normalised/weatherAUS_N.csv

In [15]:
!mv /home/normalised/weatherAUS_N.csv /home/weatherAUS_N.csv

In [16]:
!wc -l   weatherAUS_N.csv
!wc -l   weatherAUS.csv

142194 weatherAUS_N.csv

142194 weatherAUS.csv



In [17]:
!rm -r /home/normalised/

<font color='red'>
YURY=====================================      MY CODE ENDS     =====================================
 </font>


docker cp  bd_cont:/home/weatherAUS_N.csv /home/yury/weatherAUS_N.csv

docker cp  bd_cont:/home/normalise.ipynb /home/yury/normalise.ipynb
